In [ ]:
#| default_exp core

# module name here

> API details.

In [ ]:
#|hide
from nbprocess.showdoc import *

In [ ]:
#|export
import os,json
from fastcore.utils import *

In [ ]:
#|export
iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')

if iskaggle:
    from kaggle_secrets import UserSecretsClient
    sec = UserSecretsClient()
    os.environ['KAGGLE_USERNAME'] = sec.get_secret("kaggle_username")
    if not os.environ['KAGGLE_USERNAME']: raise Exception("Please insert your Kaggle username and key into Kaggle secrets")
    os.environ['KAGGLE_KEY'] = sec.get_secret("kaggle_key")

from kaggle import api

In [ ]:
#|export
def setup_comp(competition, install=''):
    "Get a path to data for `competition`, downloading it if needed"
    if iskaggle:
        if install:
            os.system(f'pip install -Uqq {install}')
        return Path('../input')/competition
    else:
        path = Path(competition)
        if not path.exists():
            import zipfile
            api.competition_download_cli(str(competition))
            zipfile.ZipFile(f'{comp}.zip').extractall(str(competition))
        return path

In [ ]:
setup_comp('titanic')

Path('titanic')

If you pass a list of space separated modules to `install`, they'll be installed if running on Kaggle.

In [ ]:
#|export
def nb_meta(user, id, title, file, competition=None, private=True, gpu=False, internet=True):
    d = {
      "id": f"{user}/{id}",
      "title": title,
      "code_file": file,
      "language": "python",
      "kernel_type": "notebook",
      "is_private": private,
      "enable_gpu": gpu,
      "enable_internet": internet,
      "keywords": [],
      "dataset_sources": [],
      "kernel_sources": []
    }
    if competition: d["competition_sources"] = [f"competitions/{competition}"]
    return d

In [ ]:
nb_meta('jhoward', 'my-notebook', 'My notebook', 'my-notebook.ipynb', competition='paddy-disease-classification')

{'id': 'jhoward/my-notebook',
 'title': 'My notebook',
 'code_file': 'my-notebook.ipynb',
 'language': 'python',
 'kernel_type': 'notebook',
 'is_private': True,
 'enable_gpu': False,
 'enable_internet': True,
 'keywords': [],
 'dataset_sources': [],
 'kernel_sources': [],
 'competition_sources': ['competitions/paddy-disease-classification']}

In [ ]:
#|export
def push_notebook(user, id, title, file, path='.', competition=None, private=True, gpu=False, internet=True):
    meta = nb_meta(user, id, title, file=file, competition=competition, private=private, gpu=gpu, internet=internet)
    path = Path(path)
    nm = 'kernel-metadata.json'
    path.mkdir(exist_ok=True, parents=True)
    with open(path/nm, 'w') as f: json.dump(meta, f, indent=2)
    api.kernels_push_cli(str(path))

## Export -

In [ ]:
#|hide
#|eval: false
from nbprocess.doclinks import nbprocess_export
nbprocess_export()